## Ⅰ. 모델링

#### 0. 라이브러리 및 데이터 import

In [41]:
# gpu 항상 사용 설정
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [42]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from keras.layers import Input
from keras.callbacks import EarlyStopping
# from keras.preprocessing import timeseries_dataset_from_array


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing import timeseries_dataset_from_array
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Input, LSTM, Dense

In [43]:
rdf=pd.read_csv('output/daegu_final.csv')

#### 데이터 전처리

In [44]:
df=rdf[:]
print('Number of rows and columns:', df.shape)

# region을 숫자로 분류
fact = pd.factorize(df['region'])
df['region'] = fact[0]

df.drop(['lightning','condition','datetime','HI','DI'], axis=1, inplace=True)
df.dropna(inplace=True)

# datetime으로 변경하기 위해 자릿수 설정
df['date'] = pd.to_datetime((df['year']*10000 + df['month']*100 + df['day']).astype(str), format='%Y%m%d')

# df.drop(['year','month','day'],axis=1,inplace=True)

df=df[['date','month','temp','rainfall','humidity','wind_speed','wind_direction','region','hour']]
# df=df[['date','temp','rainfall','humidity','wind_speed','wind_direction','region','hour']]
df

Number of rows and columns: (14831616, 15)


,date,month,temp,rainfall,humidity,wind_speed,wind_direction,region,hour
4519,2010-12-06,12,11.6,0.0,32.0,4.1,304.0,0,7
4520,2010-12-06,12,10.6,0.0,35.0,3.6,300.0,0,8
4521,2010-12-06,12,9.2,0.0,27.0,3.8,295.0,0,9
4522,2010-12-06,12,7.9,0.0,34.0,4.1,281.0,0,10
4523,2010-12-06,12,5.7,0.0,44.0,3.6,298.0,0,11
...,...,...,...,...,...,...,...,...,...
14831611,2022-04-30,4,10.7,0.0,71.0,2.1,335.0,141,19
14831612,2022-04-30,4,11.0,0.0,65.0,2.2,339.0,141,20
14831613,2022-04-30,4,11.9,0.0,61.0,2.2,338.0,141,21
14831614,2022-04-30,4,13.4,0.0,53.0,2.0,351.0,141,22


In [45]:
# df_all=df[(df['month']==12)|(df['month']==1)|(df['month']==2)|(df['month']==6)|(df['month']==7)|(df['month']==8)]
df_all=df[(df['month']==6)|(df['month']==7)|(df['month']==8)|(df['month']==9)]
df_all

,date,month,temp,rainfall,humidity,wind_speed,wind_direction,region,hour
8760,2011-06-01,6,19.0,0.0,77.0,1.6,86.0,0,0
8761,2011-06-01,6,19.8,0.0,66.0,1.3,48.0,0,1
8762,2011-06-01,6,20.4,0.0,66.0,2.1,270.0,0,2
8763,2011-06-01,6,21.9,0.0,60.0,1.6,101.0,0,3
8764,2011-06-01,6,21.8,0.0,61.0,2.2,270.0,0,4
...,...,...,...,...,...,...,...,...,...
14826523,2021-09-30,9,16.8,0.0,100.0,0.5,338.0,141,19
14826524,2021-09-30,9,17.1,0.0,100.0,1.0,328.0,141,20
14826525,2021-09-30,9,16.9,0.0,100.0,2.0,346.0,141,21
14826526,2021-09-30,9,17.5,0.0,100.0,2.2,325.0,141,22


In [46]:
# df_all_10r=df_all[(df_all['region']==10)]
# df_all_10r

In [47]:
df_all_ind=df_all.reset_index(drop=True)

In [48]:
# temp 분포 확인

# plt.figure(figsize=(30, 6))
# df_all.plot(y=['temp'])

In [49]:
df_all_ind.drop(['month'],axis=1,inplace=True)
df_all_ind

,date,temp,rainfall,humidity,wind_speed,wind_direction,region,hour
0,2011-06-01,19.0,0.0,77.0,1.6,86.0,0,0
1,2011-06-01,19.8,0.0,66.0,1.3,48.0,0,1
2,2011-06-01,20.4,0.0,66.0,2.1,270.0,0,2
3,2011-06-01,21.9,0.0,60.0,1.6,101.0,0,3
4,2011-06-01,21.8,0.0,61.0,2.2,270.0,0,4
...,...,...,...,...,...,...,...,...
4573531,2021-09-30,16.8,0.0,100.0,0.5,338.0,141,19
4573532,2021-09-30,17.1,0.0,100.0,1.0,328.0,141,20
4573533,2021-09-30,16.9,0.0,100.0,2.0,346.0,141,21
4573534,2021-09-30,17.5,0.0,100.0,2.2,325.0,141,22


#### train과 test로 데이터 분리

In [50]:
# Setting
train_split_idx=10000                 # 2020-01-01 23:00:00 행 인덱스의 번호
window_size=100                       # 과거 100시간 동안 시계열 데이터를 학습 데이터로 사용
future=1                              # 1시간 이후의 타깃 예측

# Features
X_train=df_all_ind.iloc[:train_split_idx-window_size-future,0:]  # row 0~9899

# Targets
y_train=df_all_ind.iloc[window_size+future:train_split_idx,[1]]  # 'temp' 열  # row 101~10000

print(X_train.shape,y_train.shape)

(9899, 8) (9899, 1)


In [51]:
df_all_ind.iloc[[train_split_idx-1,train_split_idx,train_split_idx+1]]

,date,temp,rainfall,humidity,wind_speed,wind_direction,region,hour
9999,2014-07-21,28.6,0.0,54.0,2.0,270.0,0,15
10000,2014-07-21,27.3,0.0,68.0,2.0,291.0,0,16
10001,2014-07-21,26.5,0.0,75.0,2.0,292.0,0,17


In [52]:
df_all_ind.shape

(4573536, 8)

In [53]:
# X_test
test_start=train_split_idx-window_size-future       # 테스트 데이터 시작 행 # row 9899
test_end=df_all_ind.shape[0]-window_size-future     # 테스트 데이터 끝 행 # row 4573435
X_test=df_all_ind.iloc[test_start:test_end,0:]      # row 9899~4573435

# y_test
# label_start= +future        # 테스트 데이터의 첫 번째 타깃 데이터 위치
y_test=df_all_ind.iloc[train_split_idx:,[1]]    # 'temp' 열 선택 10000~4573435

print(X_test.shape,y_test.shape)

(4563536, 8) (4563536, 1)


#### 입력 데이터 0~1로 조정

In [54]:
# Feature Scaling
X_train_scaled=X_train.loc[:,'temp':]
X_test_scaled=X_test.loc[:,'temp':]

scaler=MinMaxScaler()
scaler.fit(X_train_scaled.values)
X_train_scaled.loc[:,:]=scaler.transform(X_train_scaled.values)
X_test_scaled.loc[:,:]=scaler.transform(X_test_scaled.values)

#### 배치 사이즈 조정

In [55]:
# Mini Batch 크기로 시계열 변환
train_data=timeseries_dataset_from_array(X_train_scaled,y_train,sequence_length=window_size, batch_size=4096)
test_data=timeseries_dataset_from_array(X_test_scaled,y_test,sequence_length=window_size, batch_size=4096)

print(train_data)
print(test_data)

<BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>


#### 5층 레이어

In [56]:
model=Sequential()
model.add(Input(shape=[100,7]))

model.add(LSTM(units=64,return_sequences=False))
model.add(Dense(units=32,activation='linear'))
model.add(Dense(units=16,activation='linear'))
model.add(Dense(units=8,activation='linear'))
model.add(Dense(units=1,activation='linear'))

model.compile(optimizer='adam',loss='mae',metrics=['mse'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 64)                18432     
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 16)                528       
                                                                 
 dense_18 (Dense)            (None, 8)                 136       
                                                                 
 dense_19 (Dense)            (None, 1)                 9         
                                                                 
Total params: 21,185
Trainable params: 21,185
Non-trainable params: 0
_________________________________________________________________


### * GPU 사용 설정
##### 참고링크 : https://doitgrow.com/28

In [60]:
# 그래픽카드 인식 확인
!nvidia-smi

Tue May 31 17:44:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.95       Driver Version: 512.95       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8    17W /  N/A |   6416MiB /  8192MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [61]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

In [69]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices() 

# cmd와 jupyter notebook cell에서 모두 True값이 나와야 함

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9673549909258601809
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5757075456
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4304769243872247988
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [70]:
tf.test.is_gpu_available()

True

In [66]:
import tensorflow as tf 
tf.__version__ # 버전 확인, '2.1.0' 



'2.9.1'

In [58]:
# 사용가능한 CPU와 GPU 확인
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16886344172678707246
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5757075456
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14196388614869068483
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [59]:
# 모델 훈련
with tf.device("/device:GPU:0"):    # GPU를 이용한 학습
     history=model.fit(train_data,epochs=5,validation_data=test_data,verbose=1)

# 손실 함수 그래프
def plot_loss_curve(history,total_epoch=10,start=1):
    plt.figure(figsize=(5,5))
    plt.plot(range(start,total_epoch+1),
        history.history['loss'][start-1:total_epoch],
        label='Train')
    plt.plot(range(start,total_epoch+1),
        history.history['val_loss'][start-1:total_epoch],
        label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('mae')
    plt.legend()
    plt.show()

plot_loss_curve(history=history,total_epoch=len(history.history['loss']),start=1)

Epoch 1/5
3/3 [==============================] - 57s 28s/step - loss: 25.5912 - mse: 673.4990 - val_loss: 25.7929 - val_mse: 685.7400
Epoch 2/5
3/3 [==============================] - ETA: 0s - loss: 25.0443 - mse: 645.6339

KeyboardInterrupt: 